In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
from lib import jsoncorpus, datastuff
import gensim
import traceback
import numpy as np
import matplotlib.pyplot as plt
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.cross_validation
import sklearn.ensemble
import sklearn.svm
import sklearn.decomposition
from lib.scikitComponents import *
import itertools
import collections

# LDA

## Data preprocessing

In [5]:
# Load in the previously created corpus and dictionary of scraped sites
dictionary, corpus, meta_corpus, dmoz_data = jsoncorpus.load_or_create('docs/sites.jl')

Done.


In [6]:
# Build a list of all topics
allcategories = set(tuple(t) for t in dmoz_data['dmoz_categories'])
# Build a list of all top-level topics
topcategories = set(topic[0] for topic in dmoz_data['dmoz_categories'])
# Link topics to URLs
meta = list(zip(dmoz_data['urls'], dmoz_data['dmoz_categories']))
# Represent the topics in an alternative way
heirarchal_categories = lambda max_depth: [['; '.join(topics[:ti+1]) for ti, t in enumerate(topics) if ti < max_depth] for topics in dmoz_data['dmoz_categories']]
# Top categories
top_categories = [x[0] for x in heirarchal_categories(1)]

## Define a classification pipeline for the corpus data

The Pipeline() object chains together objects from the lib.scikitComponents file, so that they can be used as part of a scikit-learn classification pipeline

In [12]:
def lda_feature_pipeline(num_topics):
    return sklearn.pipeline.Pipeline([
            ('lda_model', LDAModel(dictionary, num_topics)),
            ('matrix_builder', TopicMatrixBuilder(num_topics))
            ])

In [13]:
pipe = lda_feature_pipeline(10)
pipe.fit(meta_corpus)
pipe.transform(meta_corpus)


array([[ 0.        ,  0.        ,  0.62762225, ...,  0.        ,
         0.        ,  0.35605117],
       [ 0.7579478 ,  0.08026194,  0.03266862, ...,  0.06515138,
         0.        ,  0.05920785],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.0532856 ],
       ..., 
       [ 0.05000009,  0.05000007,  0.54999969, ...,  0.05000001,
         0.05000004,  0.05      ],
       [ 0.        ,  0.41597238,  0.53402742, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.05299889,  0.05262323,  0.19986479, ...,  0.        ,
         0.05347456,  0.05267389]])

Define a classifier (decision trees), and chain the preprocessing step to the classifier

In [16]:
classifier = sklearn.ensemble.ExtraTreesClassifier(random_state=0, n_estimators=100, oob_score=True, bootstrap=True, n_jobs=4)
#classifier = sklearn.svm.SVC()
clf = sklearn.pipeline.Pipeline([
    ('preprocess_meta', lda_feature_pipeline(len(topcategories))),
    ('classification', classifier)])


In [ ]:
clf = sklearn.pipeline.Pipeline([
    ('lda_model', LDAModel(dictionary, num_topics)),
    ('matrix_builder', TopicMatrixBuilder(num_topics)),
    ('classification', classifier)])


In [19]:
#matrix = lda_feature_pipeline(len(top_categories)).fit_transform(np.array(meta_corpus))

In [20]:
#pca = sklearn.decomposition.PCA(n_components=0.9)
#pca.fit_transform(np.array(matrix))
#[[dictionary[w] for w in np.where(np.abs(comp)>0.08)[0]] for comp in pca.components_]

## Process the dmoz categories
In this case, we will just turn the top category in to an index

In [23]:
dmoz_encoder = sklearn.preprocessing.LabelEncoder().fit(top_categories)
classes = dmoz_encoder.transform(top_categories)
print classes

[7 7 7 ..., 3 3 3]


## Run the classifier

We can use the meta corpus or the full body corpus here, just by replacing a single argument

The data is split in to training and test sets, and then fit to the training set. The LDA model is generated ***only*** from the training set, not the testing set.

In [24]:
sklearn.cross_validation.cross_val_score(clf, meta_corpus, classes)

array([ 0.17018634,  0.1689612 ,  0.15365239])

In [37]:
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(np.array(meta_corpus), classes)

In [38]:
model = clf.fit(X_train, y_train)
print model.score(X_test, y_test)

0.42


In [45]:
category_scores = collections.defaultdict(float)
category_counts = collections.defaultdict(float)
for real, pred in sorted(zip(dmoz_encoder.inverse_transform(y_test), dmoz_encoder.inverse_transform(model.predict(X_test)))):
    category_scores[real] += 1 if real == pred else 0
    category_counts[real] += 1 

### Find the score for each category. 

Note that if the classifier assigned *random* categories, the score would be $\frac{1}{\textrm{num categories}}$. Instead, it is actually quite high in some casses, indicating a moderate ammount of succcess (given how naive this is!)

In [46]:
print "Category       Proportion   Score"
print "---------------------------------"
for cat, proportion, score in [(k, category_counts[k]/len(y_test), category_scores[k] / category_counts[k]) for k in category_counts.keys()]:
    print "{:<15} {:<7.4}      {:<5.4}".format(cat, proportion, score)

Category       Proportion   Score
---------------------------------
arts            0.06         0.3611
shopping        0.05333      0.5625
reference       0.07167      0.3721
business        0.08333      0.44 
kids and teens  0.02333      0.1429
computers       0.09833      0.5424
recreation      0.06667      0.3  
sports          0.1683       0.7525
society         0.08         0.2083
health          0.05333      0.375
home            0.06167      0.3243
games           0.045        0.2593
news            0.05167      0.2581
science         0.08333      0.24 


In [53]:
dmoz_data['meta']

[{u'author': [u'abc', u'news'],
  u'description': [u'get',
   u'breaking',
   u'national',
   u'and',
   u'world',
   u'news',
   u'broadcast',
   u'video',
   u'coverage',
   u'and',
   u'exclusive',
   u'interviews',
   u'find',
   u'the',
   u'top',
   u'news',
   u'online',
   u'at',
   u'abc',
   u'news'],
  u'keywords': [u'abc',
   u'news',
   u'breaking',
   u'news',
   u'headline',
   u'news',
   u'latest',
   u'news',
   u'top',
   u'news',
   u'online',
   u'news',
   u'video',
   u'news',
   u'world',
   u'news',
   u'national',
   u'news',
   u'abc',
   u'news',
   u'abcnews',
   u'abc',
   u'news',
   u'online'],
  u'title': [u'abc',
   u'news',
   u'breaking',
   u'news',
   u'latest',
   u'news',
   u'-',
   u'abc',
   u'news']},
 {u'author': [u'british', u'path'],
  u'description': [u'british',
   u'pathe',
   u'the',
   u'worlds',
   u'leading',
   u'multimedia',
   u'resource',
   u'with',
   u'a',
   u'history',
   u'stretching',
   u'back',
   u'over',
   u'a',
   u

In [51]:
print meta[20]
[dictionary[x] for x, y, in meta_corpus[20]]

(u'http://www.allyoucanread.com/', [u'news', u'directories'])


[u'magazines',
 u'massive',
 u'sites',
 u'world',
 u'directory',
 u'media',
 u'international',
 u'countries',
 u'news',
 u'newspapers',
 u'local']